In [2]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Dataset

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.19.0


In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.utils import normalize

import numpy as np
import os
import string

In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: sentencecompletion
Azure region: westeurope
Subscription id: 5334814e-153c-45f0-9557-00aa820611b9
Resource group: SentenceCompletionAzure


In [4]:
datastore = ws.get_default_datastore()
dataset = Dataset.get_by_name(ws, name='Korpus-segmentation')
mount_ctx0 = dataset.mount()  
mount_ctx0.start() 
dataset_mount_corpora = mount_ctx0.mount_point
print(dataset_mount_corpora)
corpora_files = os.listdir(dataset_mount_corpora)
print(corpora_files)

dataset = Dataset.get_by_name(ws, name='word2vec-wiki-nkfp-polish')
mount_ctx1 = dataset.mount()  
mount_ctx1.start() 
dataset_mount_word2vec = mount_ctx1.mount_point
print(dataset_mount_word2vec)
w2v_files = os.listdir(dataset_mount_word2vec)
print(w2v_files)

/tmp/tmpavb5shad
['1.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt', '8.txt', '9.txt']
/tmp/tmp_wcdvxsy
['nkjp+wiki-forms-all-100-cbow-hs.txt']


In [24]:
texts = []
for file_ in corpora_files[:2]:
    texts.append(open(os.path.join(dataset_mount_corpora, file_), 'rb').read().decode("utf-8")[:100000])
    l = (len(texts[-1]))
    print(f"Length of text {file_}: {l} characters")

texts[0][500:1000]

Length of text 1.txt: 100000 characters
Length of text 2.txt: 100000 characters


'tanisław janicki , minister pracy i opieki społecznej jerzy iwanowski , minister poczt i telegrafów hubert linde , minister kultury i sztuki zenon przesmycki , minister robót publicznych józef próchnik , kierownik ministerstwa komunikacji julian eberhardt , minister skarbu józef englich , minister aprowizacji antoni minkiewicz , minister zdrowia publicznego tomasz janiszewski , kierownik ministerstwa spraw wojskowych pułk .\r\njan wroczyński .\r\npodsekretarze stanu :\r\nw min . spraw zewnętrznych wła'

In [25]:
for i, text in enumerate(texts):
  text = text.lower()
  vocab = sorted(set(text))
  polish_alphabet = "aąbcćdeęfghijklłmnńoóprsśtuwyzźż"

  unwanted_whitespaces = string.whitespace.translate(str.maketrans("", "", " "))
  allowed_chars = polish_alphabet + polish_alphabet.upper() + string.digits + "!?.,:%" + " "
  unwanted_chars = "".join(vocab).translate(str.maketrans("","",allowed_chars))
  
  print(f"--------- {i}/{len(texts)} ----------")
  text = text.replace("\r\n", " ")
  text = text.translate(str.maketrans(unwanted_whitespaces, "".join([" " for _ in range(len(unwanted_whitespaces))])))
  text = text.translate(str.maketrans("", "", unwanted_chars))
  texts[i] = text

texts[0]

--------- 0/2 ----------
--------- 1/2 ----------


'sprawozdanie stenograficzne z 1 . posiedzenia sejmu ustawodawczego z dnia 10 . lutego 1919 r .  początek posiedzenia o godzinie 11 minut 45 .  ze strony rządu obecni : prezydent ministrów i minister spraw zagranicznych ignacy jan paderewski , minister spraw wewnętrznych stanisław wojciechowski , minister sprawiedliwości leon supiński , minister wyznań religijnych i oświecenia publicznego jan łukasiewicz , minister przymysłu i handlu kazimierz hącia , minister rolnictwa i dóbr państwowych stanisław janicki , minister pracy i opieki społecznej jerzy iwanowski , minister poczt i telegrafów hubert linde , minister kultury i sztuki zenon przesmycki , minister robót publicznych józef próchnik , kierownik ministerstwa komunikacji julian eberhardt , minister skarbu józef englich , minister aprowizacji antoni minkiewicz , minister zdrowia publicznego tomasz janiszewski , kierownik ministerstwa spraw wojskowych pułk . jan wroczyński . podsekretarze stanu : w min . spraw zewnętrznych władysław w

In [58]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(standardize=None)
text_ds = tf.data.Dataset.from_tensor_slices(texts[0].split()).batch(128)
vectorizer.adapt(text_ds)

In [66]:
len(vectorizer.get_vocabulary())

6707

In [51]:
from gensim import models
path = os.path.join(dataset_mount_word2vec, 'nkjp+wiki-forms-all-100-cbow-hs.txt')

word2vec_model = models.KeyedVectors.load_word2vec_format(path)

embedding_matrix = word2vec_model.vectors
print('Shape of embedding matrix: ', embedding_matrix.shape)

In [76]:
word2vec_model["xd"]

array([ 0.497415,  0.432435,  0.944955,  1.844033,  0.63157 ,  0.952579,
       -0.755004, -1.521198, -1.29483 , -1.5341  , -0.892172,  0.159664,
        0.21409 ,  1.079934, -1.308787,  0.551752, -0.585167, -0.204652,
        0.51532 ,  0.041035,  1.225714, -0.258516, -0.277476,  1.419884,
        0.700549, -1.66024 ,  0.459371, -0.190116, -0.417183, -0.278603,
        0.287218,  1.599751, -1.315916, -2.288384,  0.971492, -2.516749,
       -0.358711,  2.800197, -0.693089,  2.088943, -1.456914, -0.750984,
       -1.848792, -0.264186, -0.469719, -1.591162,  1.188315, -0.35073 ,
       -1.168758, -3.706449, -0.568401,  0.618599, -0.55414 ,  0.157055,
        0.62748 ,  0.569068,  0.080014, -0.697131, -1.055869,  0.104964,
       -0.262386,  0.240416, -0.487353,  0.159707, -0.144954, -1.557009,
        0.043999, -1.346868,  2.00409 ,  0.139921, -1.949323, -0.120989,
       -0.851479, -0.312084, -0.062562,  2.626721, -0.039137, -0.636441,
       -0.028918, -0.944736, -0.260646, -0.616735, 

In [65]:
vocab = set()
for text in texts:
    vocab = vocab.union(set(text.split()))
vocab = sorted(vocab)

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(standardize=None)
text_ds = tf.data.Dataset.from_tensor_slices(vocab).batch(128)
vectorizer.adapt(text_ds)


In [67]:
vectorizer.get_vocabulary()[:100]

['',
 '[UNK]',
 'żądać',
 'żądanie',
 'żąda',
 'żyć',
 'żywotnych',
 'żywotności',
 'żywot',
 'żywność',
 'żyjących',
 'żyje',
 'żydów',
 'żydowskiej',
 'żydowskiego',
 'żydowskie',
 'żydowskich',
 'żydowska',
 'żydostwa',
 'życząc',
 'życzy',
 'życzliwości',
 'życzeń',
 'życzeniem',
 'życiu',
 'życie',
 'życia',
 'żuławski',
 'żukowie',
 'żołnierzy',
 'żołnierzowi',
 'żołnierzem',
 'żołnierze',
 'żołnierzach',
 'żołnierza',
 'żołnierz',
 'żołnierskiej',
 'żeglugi',
 'żeby',
 'żebrowski',
 'że',
 'żałuski',
 'żadną',
 'żadnym',
 'żadnych',
 'żadnej',
 'żadnego',
 'żadne',
 'żaden',
 'ż',
 'źródło',
 'źródłem',
 'źle',
 'święto',
 'świetności',
 'świetnej',
 'świecie',
 'światu',
 'światową',
 'światopełk',
 'światem',
 'świata',
 'świat',
 'świadomość',
 'świadków',
 'świadka',
 'świadectwem',
 'świadectw',
 'środków',
 'środkową',
 'środek',
 'średnią',
 'średniawskiego',
 'średnia',
 'śnieg',
 'śmy',
 'ślązka',
 'ślązk',
 'śląsku',
 'śląskiem',
 'śląskiej',
 'śląskich',
 'śląska',
 '

In [68]:
len(vocab)

6705

In [69]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   1,    1,    1, 3893,    1,    1])

In [70]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

test = ["pan", "jest", "tu", "i", "tam", "."]
[word_index[w] for w in test]

[3706, 5262, 1607, 5421, 1728, 6703]

In [82]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = None
    try:
        embedding_vector = word2vec_model[word]
    except Exception:
        pass
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 6339 words (368 misses)


In [14]:
all_tokens = []
tokens_datasets = []
for i, text in enumerate(texts):
  print(f"Tokenizacja tekstu {i}/{len(texts)}")
  tokens = tokens_from_words(text.split())
  all_tokens.append(tokens)
  tokens_datasets.append(tf.data.Dataset.from_tensor_slices(tokens))

for token in tokens_datasets[0].take(10):
    print(token)
    print(words_from_tokens(token).numpy().decode('utf-8'))

Tokenizacja tekstu 0/1
tf.Tensor(4275, shape=(), dtype=int64)
sprawozdanie
tf.Tensor(474424, shape=(), dtype=int64)
stenograficzne
tf.Tensor(5, shape=(), dtype=int64)
z
tf.Tensor(29, shape=(), dtype=int64)
1
tf.Tensor(1, shape=(), dtype=int64)
[UNK]
tf.Tensor(4330, shape=(), dtype=int64)
posiedzenia
tf.Tensor(1071, shape=(), dtype=int64)
sejmu
tf.Tensor(64026, shape=(), dtype=int64)
ustawodawczego
tf.Tensor(5, shape=(), dtype=int64)
z
tf.Tensor(200, shape=(), dtype=int64)
dnia


In [15]:
seq_length = 20
examples_per_epoch = len(texts[0])//(seq_length+1)
sequences_d = []
for i, tokens_dataset in enumerate(tokens_datasets):
  print(f"Tworzenie zbiorów sekwencji {i}/{len(tokens_dataset)}")
  sequences_d.append(tokens_dataset.batch(seq_length+1, drop_remainder=True))

for seq in sequences_d[0].take(10):
  print(seq)
  print((text_from_tokens(seq).numpy()).decode("utf-8"))

Tworzenie zbiorów sekwencji 0/17370
tf.Tensor(
[  4275 474424      5     29      1   4330   1071  64026      5    200
     86      1    576   3973     73      1    991   4330     13   1321
    147], shape=(21,), dtype=int64)
sprawozdaniestenograficznez1[UNK]posiedzeniasejmuustawodawczegozdnia10[UNK]lutego1919r[UNK]początekposiedzeniaogodzinie11
tf.Tensor(
[ 826  623    1   36  221  740 9105    1  660 1846    3  618  529 1638
    1    1    1    1  618  529 2430], shape=(21,), dtype=int64)
minut45[UNK]zestronyrząduobecni[UNK]prezydentministrówiministersprawzagranicznych[UNK][UNK][UNK][UNK]ministersprawwewnętrznych
tf.Tensor(
[      1   70663       1     618    1516       1       1       1     618
   13803    4980       3   11230    2301       1       1       1     618
 1451485       3    2506], shape=(21,), dtype=int64)
[UNK]wojciechowski[UNK]ministersprawiedliwości[UNK][UNK][UNK]ministerwyznańreligijnychioświeceniapublicznego[UNK][UNK][UNK]ministerprzymysłuihandlu
tf.Tensor(
[    1     

In [16]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

datasets = []
for i, sequences in enumerate(sequences_d):
  print(f"{i}/{len(sequences_d)}")
  datasets.append(sequences_d[i].map(split_input_target))

for input_example, target_example in  datasets[0].take(20):
  print("Input :", (text_from_tokens(input_example).numpy()).decode('utf-8'))
  print("Target:", (text_from_tokens(target_example).numpy()).decode('utf-8'))

0/1
Input : sprawozdaniestenograficznez1[UNK]posiedzeniasejmuustawodawczegozdnia10[UNK]lutego1919r[UNK]początekposiedzeniaogodzinie
Target: stenograficznez1[UNK]posiedzeniasejmuustawodawczegozdnia10[UNK]lutego1919r[UNK]początekposiedzeniaogodzinie11
Input : minut45[UNK]zestronyrząduobecni[UNK]prezydentministrówiministersprawzagranicznych[UNK][UNK][UNK][UNK]ministerspraw
Target: 45[UNK]zestronyrząduobecni[UNK]prezydentministrówiministersprawzagranicznych[UNK][UNK][UNK][UNK]ministersprawwewnętrznych
Input : [UNK]wojciechowski[UNK]ministersprawiedliwości[UNK][UNK][UNK]ministerwyznańreligijnychioświeceniapublicznego[UNK][UNK][UNK]ministerprzymysłui
Target: wojciechowski[UNK]ministersprawiedliwości[UNK][UNK][UNK]ministerwyznańreligijnychioświeceniapublicznego[UNK][UNK][UNK]ministerprzymysłuihandlu
Input : [UNK][UNK][UNK]ministerrolnictwaidóbrpaństwowych[UNK][UNK][UNK]ministerpracyiopiekispołecznej[UNK][UNK][UNK]minister
Target: [UNK][UNK]ministerrolnictwaidóbrpaństwowych[UNK][UNK][UNK]minis

In [17]:
BATCH_SIZE = 128

BUFFER_SIZE = 10000

dataset_train = datasets[0]
for dataset in datasets[1:]:
  dataset_train = dataset_train.concatenate(dataset)

dataset_train = (
    dataset_train.shuffle(BUFFER_SIZE)
                .batch(BATCH_SIZE, drop_remainder=True)
                .prefetch(tf.data.experimental.AUTOTUNE))

In [18]:
len(dataset_train)

6

In [25]:
# Budowanie modelu
print("---------- Budowanie modelu ---------")

vocab_size = len(vocab)
print(vocab_size)


units = 512

class MyModelLSTM(tf.keras.Model):
    def __init__(self, vocab_size, embedding_matrix, units):
        super(MyModelLSTM, self).__init__()
        self.embedding = tf.keras.layers.Embedding(embedding_matrix.shape[0],
                                                    embedding_matrix.shape[1],
                                                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                                    trainable=False)
        self.lstm1 = tf.keras.layers.LSTM(units,
                                            return_sequences=True,
                                            return_state=True,
                                            dropout=0.2)
        self.lstm2 = tf.keras.layers.LSTM(units,
                                            return_sequences=True,
                                            return_state=True,
                                            dropout=0.2)
        self.dense = tf.keras.layers.Dense(embedding_matrix.shape[0])

    def call(self, inputs, states1=None, states2=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x)
        if states1 is None:
            state1, state2 = self.lstm1.get_initial_state(x)
            states1 = [state1, state2]
        x, state1, state2 = self.lstm1(x, initial_state=states1, training=training)
        states1 = [state1, state2]
        if states2 is None:
            state2, state2 = self.lstm2.get_initial_state(x)
            states2 = [state1, state2]
        x, state1, state2 = self.lstm2(x, initial_state=states2, training=training)
        states2 = [state1, state2]
        x = self.dense(x, training=training)

        if return_state:
            return x, states1, states2
        else: 
            return x



---------- Budowanie modelu ---------
4119


In [26]:
model = MyModelLSTM(
    vocab_size=len(tokens_from_words.get_vocabulary()),
    embedding_matrix=embedding_matrix,
    units=units)

for input_example_batch, target_example_batch in dataset_train.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

#model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[512,2123132] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)


tf.exp(mean_loss).numpy()


model.compile(optimizer='adam', loss=loss)